In [2]:
import pandas as pd
import numpy as np
import json

#cargo los datasets
#si agrego  dtype='unicode' evito algunos errores sobre tipos de datos
dfMovies = pd.read_csv(r'datasets\movies_dataset.csv', dtype='unicode')
dfCredits= pd.read_csv(r'datasets\credits.csv', dtype='unicode')

pd.set_option("display.max_rows", 2000, "display.max_columns", None)

In [12]:
#FUNCIONES

#desanida columna y devuelve una lista de diccionarios
def desanidarColumna(columna):
    lista = []
    listaDict=[]
    counter=0
    for k, v in columna.items():
        if isinstance(v, str):
            lista.append(v)
        else:
            lista.append('valor desconocido')
    for i in lista:   
                try:
                    i=json.loads(lista[counter].replace("'", '"'))
                    listaDict.append(i)
                    counter+=1
                except:
                    None
                    counter+=1
    listaDict=pd.json_normalize(listaDict)
    return listaDict
    


In [15]:
#desanido belongs to collection
dfPeliculas = (desanidarColumna(dfMovies['belongs_to_collection']))
dfPeliculas

id               3
name             3
poster_path      3
backdrop_path    3
dtype: int64

In [16]:
#la columna generos hay que desanidarle cada columna despues de aplciarle la funcion
dfGeneros = (desanidarColumna(dfMovies['genres']))
dfGeneros


,0,1,2,3,4,5,6,7
0,"{'id': 16, 'name': 'Animation'}","{'id': 35, 'name': 'Comedy'}","{'id': 10751, 'name': 'Family'}",None,None,None,None,None
1,"{'id': 12, 'name': 'Adventure'}","{'id': 14, 'name': 'Fantasy'}","{'id': 10751, 'name': 'Family'}",None,None,None,None,None
2,"{'id': 10749, 'name': 'Romance'}","{'id': 35, 'name': 'Comedy'}",None,None,None,None,None,None
3,"{'id': 35, 'name': 'Comedy'}","{'id': 18, 'name': 'Drama'}","{'id': 10749, 'name': 'Romance'}",None,None,None,None,None
4,"{'id': 35, 'name': 'Comedy'}",None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
45461,"{'id': 18, 'name': 'Drama'}","{'id': 10751, 'name': 'Family'}",None,None,None,None,None,None
45462,"{'id': 18, 'name': 'Drama'}",None,None,None,None,None,None,None
45463,"{'id': 28, 'name': 'Action'}","{'id': 18, 'name': 'Drama'}","{'id': 53, 'name': 'Thriller'}",None,None,None,None,None
45464,None,None,None,None,None,None,None,None


# Convertimos los datos anidados en diccionarios cargando las cadenas JSON
def convert_to_dict(json_string):
    try:
        return json.loads(json_string.replace("'", '"'))
    except:
        return None



   
columna_dict = belongs_to_collection.apply(convert_to_dict)

# Filtramos los valores que no sean 'None'
valid_columna = columna_dict.dropna()

# Desanidamos los datos usando json_normalize
columna_df = pd.json_normalize(valid_columna)

# Visualizamos el DataFrame resultante
columna_df.head()